In [1]:
import drawsvg as draw
import math
import os

In [2]:
# Create output directory for SVGs
output_dir = 'svg_tiles'
os.makedirs(output_dir, exist_ok=True)

# Parameters
num_rings = 16
num_radial_divisions = 24
radius_step = 10  # Distance between each ring

# Set the minimum inner radius for the entire disk to 10
min_inner_radius = 10

# Adjust the logic for all rings
for i in range(num_rings):
    for j in range(num_radial_divisions):
        # Create a new drawing for each tile
        d = draw.Drawing(400, 400, origin='center')

        # Define the boundaries of the tile
        inner_radius = max(min_inner_radius, (i + 1) * radius_step)
        outer_radius = (i + 2) * radius_step
        angle_step = 2 * math.pi / num_radial_divisions
        start_angle = j * angle_step
        end_angle = (j + 1) * angle_step
        # The inner ring only has 12 divisions
        if i == 0:
            if j >= num_radial_divisions // 2:
                continue
            start_angle = j * angle_step * 2
            end_angle = (j + 1) * angle_step * 2

            

        # Draw the full bounded sector
        path = draw.Path(stroke='black', stroke_width=0.5)
        path.M(inner_radius * math.cos(start_angle), inner_radius * math.sin(start_angle))
        path.L(outer_radius * math.cos(start_angle), outer_radius * math.sin(start_angle))
        path.A(outer_radius, outer_radius, 0, 0, 1, outer_radius * math.cos(end_angle), outer_radius * math.sin(end_angle))
        path.L(inner_radius * math.cos(end_angle), inner_radius * math.sin(end_angle))
        path.A(inner_radius, inner_radius, 0, 0, 0, inner_radius * math.cos(start_angle), inner_radius * math.sin(start_angle))
        path.Z()
        d.append(path)

        # Save the drawing as an SVG
        tile_filename = f'tile_ring{i}_division{j}.svg'
        d.save_svg(os.path.join(output_dir, tile_filename))

In [3]:
# import os
# from IPython.display import SVG, display

# # Directory containing SVG files
# svg_dir = 'svg_tiles'

# # Load and display each SVG tile
# svg_files = sorted([f for f in os.listdir(svg_dir) if f.endswith('.svg')])
# for svg_file in svg_files:
#     print(f'Displaying: {svg_file}')
#     display(SVG(os.path.join(svg_dir, svg_file)))

In [4]:
template_json_text = r"""
{
  "id": 1,
  "type": "canvas",
  "title": "Panel Title",
  "gridPos": {
    "x": 0,
    "y": 0,
    "h": 8,
    "w": 12
  },
  "fieldConfig": {
    "defaults": {
      "mappings": [],
      "thresholds": {
        "mode": "absolute",
        "steps": [
          {
            "color": "blue",
            "value": null
          },
          {
            "value": 0,
            "color": "#EAB839"
          },
          {
            "value": 10,
            "color": "red"
          }
        ]
      },
      "color": {
        "mode": "thresholds"
      },
      "min": 0,
      "fieldMinMax": false,
      "max": 2
    },
    "overrides": []
  },
  "pluginVersion": "11.6.0+security-01",
  "targets": [
    {
      "refId": "A",
      "datasource": {
        "type": "prometheus",
        "uid": "EflW1u9nz"
      },
      "editorMode": "builder",
      "expr": "sphenix_sEPD_current_readback_uA",
      "legendFormat": "__auto",
      "range": true,
      "useBackend": false,
      "disableTextWrap": false,
      "fullMetaSearch": false,
      "includeNullMetadata": true
    }
  ],
  "datasource": {
    "uid": "EflW1u9nz",
    "type": "prometheus"
  },
  "options": {
    "inlineEditing": true,
    "showAdvancedTypes": true,
    "panZoom": false,
    "infinitePan": false,
    "root": {
      "type": "frame",
      "elements": [
        {
        }
      ],
      "background": {
        "color": {
          "fixed": "transparent"
        },
        "image": {
          "mode": "fixed",
          "field": "",
          "fixed": ""
        },
        "size": "cover"
      },
      "border": {
        "color": {
          "fixed": "dark-green"
        }
      },
      "name": "Element 1745423716553",
      "constraint": {
        "vertical": "top",
        "horizontal": "left"
      },
      "placement": {
        "width": 100,
        "height": 100,
        "top": 0,
        "left": 0,
        "rotation": 0
      }
    }
  }
}
"""
import json
template_json = json.loads(template_json_text)


In [5]:
import json
import os

# Load all SVG files
svg_dir = 'svg_tiles'
svg_files = sorted([f for f in os.listdir(svg_dir) if f.endswith('.svg')])

# Clear existing elements except the first one (template)
template_json['options']['root']['elements'] = template_json['options']['root']['elements'][:1]

# Add each tile as an element to the template JSON
for svg_file in svg_files:
    # Parse ring and division from filename
    parts = svg_file.replace('.svg', '').split('_')
    ring = int(parts[1].replace('ring', ''))
    division = int(parts[2].replace('division', ''))
    
    # Create element for this tile
    element = {
        "config": {
            "path": {
                "mode": "fixed",
                "fixed": f"https://raw.githubusercontent.com/tlprotzman/sepd_tile_svg/refs/heads/main/svg_tiles/{svg_file}"
            },
            "fill": {
                "fixed": "#FFFFFF",
                "field": f"{{__name__=\"sphenix_sEPD_current_readback_uA\", channel=\"{ring*24 + division}\", dev_level=\"production\", instance=\"daq02.sphenix.bnl.gov:9113\", job=\"GLOBAL\"}}"
            }
        },
        "background": {
            "color": {
                "fixed": "transparent"
            }
        },
        "placement": {
            "top": 0,
            "height": 200,
            "left": 0,
            "width": 200,
            "rotation": 0
        },
        "links": [],
        "type": "icon",
        "name": f"Tile Ring {ring} Division {division}",
        "constraint": {
            "vertical": "center",
            "horizontal": "center"
        },
        "border": {
            "color": {
                "fixed": "transparent"
            }
        }
    }
    
    template_json['options']['root']['elements'].append(element)

# Save the updated template JSON
with open('sepd_panel.json', 'w') as f:
    json.dump(template_json, f, indent=2)

print(f"Added {len(svg_files)} tiles to the JSON template. Saved as 'sepd_panel.json'")

Added 372 tiles to the JSON template. Saved as 'sepd_panel.json'
